In [1]:
import pandas as pd

In [2]:
updated_p = pd.read_csv('matched_s2_products.csv')

In [3]:
updated_p['product_id'].unique()

array(['S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE',
       'S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_20230912T173224.SAFE',
       'S2A_MSIL1C_20170928T100021_N0500_R122_T32SNC_20230912T173224.SAFE',
       ...,
       'S2A_MSIL1C_20210811T101031_N0500_R022_T32TQQ_20230211T192458.SAFE',
       'S2A_MSIL1C_20210821T101031_N0500_R022_T32TQQ_20230119T115533.SAFE',
       'S2A_MSIL1C_20210821T101031_N0500_R022_T33TUK_20230119T115533.SAFE'],
      dtype=object)

In [4]:
!pip install hypercoast
!pip install geopandas
!pip install rasterio
!pip install shapely

In [5]:
updated_p[updated_p['product_id']=='S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_20230912T173224.SAFE']

,filament_id,lat_centroid,lon_centroid,dec_time,date,product_id,datetime,mgrs_tile
1,2001,34.930481,11.026350,2017.740763,2017-09-28,S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_2...,2017-09-28T10:11:08.261Z,Unknown
2,2002,34.895719,11.048051,2017.740763,2017-09-28,S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_2...,2017-09-28T10:11:08.261Z,Unknown
3,2003,34.782466,11.085260,2017.740763,2017-09-28,S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_2...,2017-09-28T10:11:08.261Z,Unknown
4,2004,34.695855,10.773533,2017.740763,2017-09-28,S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_2...,2017-09-28T10:11:08.261Z,Unknown
5,2005,34.526897,10.594494,2017.740763,2017-09-28,S2A_MSIL1C_20170928T100021_N0500_R122_T32SPD_2...,2017-09-28T10:11:08.261Z,Unknown


In [13]:
import shutil

src = "hypercoast_work/safe/S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE.zip"
dst = "hypercoast_work/safe/"

shutil.unpack_archive(src, extract_dir=dst)


In [14]:
import pandas as pd
import os
import zipfile
import rasterio
from rasterio.windows import Window
from pathlib import Path
import hypercoast

# === Setup Paths ===
work_dir = Path("hypercoast_work")
input_dir = work_dir / "safe"
output_dir = work_dir / "output"
image_patch_dir = Path("geotiffs")
input_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)
image_patch_dir.mkdir(parents=True, exist_ok=True)

# === Download ACOLITE (once) ===
acolite_dir = hypercoast.download_acolite(str(work_dir))

# === Group by unique S2 product ===
product_id = 'S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE'
product_groups = updated_p.groupby("product_id")

for s2_product, group in product_groups:
    if s2_product != product_id:
        continue
    s2_product = s2_product.split('.')[0]
    print(f"\n🛰 Processing {s2_product} with {len(group)} filaments")

    safe_zip_path = input_dir / f"{s2_product}.zip"
    print(safe_zip_path)
    safe_folder_path = input_dir / f"{s2_product}.SAFE"

    # Step 1: Extract ZIP if not already
    if not safe_folder_path.exists():
        if safe_zip_path.exists():
            print(f"📦 Extracting {safe_zip_path}")
            try:
                with zipfile.ZipFile(safe_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(input_dir)
            except Exception as e:
                print(f"❌ Failed to extract {safe_zip_path}: {e}")
                continue
        else:
            print(f"❌ ZIP file not found: {safe_zip_path}")
            continue
    else:
        print("✅ .SAFE folder exists.")

    # Step 2: Run ACOLITE once for this product
    try:
        print(f"🚀 Running ACOLITE...")
        hypercoast.run_acolite(
            acolite_dir=acolite_dir,
            input_file=str(safe_folder_path),
            out_dir=str(output_dir),
            l2w_parameters="Rrs_*",
            resolution=10,
            rgb_rhot=True,
            map_l2w=True
        )
    except Exception as e:
        print(f"❌ ACOLITE failed for {s2_product}: {e}")
        continue

    # Step 3: For each filament in this product, crop its patch
    band_path = output_dir / f"{s2_product}_Rrs_B04.tif"
    if not band_path.exists():
        print(f"⚠️ Missing ACOLITE output: {band_path}")
        continue

    for _, filament in group.iterrows():
        filament_id = filament["filament_id"]
        x_center = int(filament["x_centroid"])
        y_center = int(filament["y_centroid"])
        output_patch_path = image_patch_dir / f"filament_{filament_id}_image.tif"

        try:
            with rasterio.open(band_path) as src:
                window = Window(x_center - 128, y_center - 128, 256, 256)
                transform = src.window_transform(window)

                profile = src.meta.copy()
                profile.update({
                    "height": 256,
                    "width": 256,
                    "transform": transform
                })

                with rasterio.open(output_patch_path, "w", **profile) as dst:
                    dst.write(src.read(window=window))
            print(f"✅ Saved patch: {output_patch_path}")
        except Exception as e:
            print(f"❌ Failed to crop filament {filament_id}: {e}")


hypercoast_work/acolite_py_mac_20231023.0.tar.gz already exists. Skip downloading.

🛰 Processing S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243 with 4 filaments
hypercoast_work/safe/S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.zip
✅ .SAFE folder exists.
🚀 Running ACOLITE...


osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
acolite/sentinel2/l1_convert.py:417: RuntimeWarning: Mean of empty slice
acolite/sentinel2/l1_convert.py:418: RuntimeWarning: Mean of empty slice
Traceback (most recent call last):
  File "launch_acolite.py", line 79, in <module>
  File "launch_acolite.py", line 73, in launch_acolite
  File "acolite/acolite/acolite_run.py", line 111, in acolite_run
  File "acolite/acolite/acolite_l1r.py", line 72, in acolite_l1r
  File "acolite/sentinel2/l1_convert.py", line 622, in l1_convert
  File "acolite/output/nc_write.py", line 253, in nc_write
  File "src/netCDF4/_netCDF4.pyx", line 5505, in netCDF4._netCDF4.Variable.__setitem__
  File "src/netCDF4/_netCDF4.pyx", line 5788, in netCDF4._netCDF4.Variable._put
  File "src/netCDF4/_netCDF4.pyx", line 2029, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: HDF error
[36

Running ACOLITE processing - Generic Version 20231023.0
Python - darwin - 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:54:14) [Clang 15.0.7 ]
Platform - Darwin 23.1.0 - x86_64 - Darwin Kernel Version 23.1.0: Mon Oct  9 21:28:12 PDT 2023; root:xnu-10002.41.9~6/RELEASE_ARM64_T8103
Run ID - 20250407_093845
Identified hypercoast_work/safe/S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE as Sentinel-2 type
Starting conversion of 1 scenes
Starting conversion of hypercoast_work/safe/S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE
Importing metadata from L1C_T32SPD_A011702_20170918T100023
Reading per pixel geometry
Computing band average per detector geometry
Detector 4
Detector 5
Detector 6
Detector 7
Detector 8
Detector 9
Wrote raa (10980, 10980)
Wrote vza (10980, 10980)
Wrote sza (10980, 10980)
Writing geolocation lon/lat
(10980, 10980)
Wrote lon
(10980, 10980)
❌ ACOLITE failed for S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243: 

In [15]:
import glob

jp2_files = glob.glob("hypercoast_work/safe/S2A_MSIL1C_20170918T100021_*.SAFE/GRANULE/*/IMG_DATA/*.jp2", recursive=True)
print(f"Found {len(jp2_files)} .jp2 files")


Found 14 .jp2 files


In [1]:
import rasterio
from rasterio.transform import from_origin
import numpy as np
from netCDF4 import Dataset
def extract_rrs_multiband_patch(nc_path, lat_center, lon_center, patch_size=256, output_path="rrs_patch.tif"):
    ds = Dataset(nc_path)
    lat = ds.variables["lat"][:]
    lon = ds.variables["lon"][:]

    # Get all available Rrs bands
    rrs_band_names = [k for k in ds.variables.keys() if k.startswith("Rrs_")]
    rrs_band_names.sort(key=lambda b: int(b.split("_")[1]))  # Sort by wavelength

    # Find center pixel
    lat_center = float(lat_center)  # Convert lat_center to float
    lon_center = float(lon_center)  # Convert lon_center to float


    dist = np.sqrt((lat - lat_center)[:, None]**2 + (lon - lon_center)[None, :]**2)
    y_center, x_center = np.unravel_index(np.argmin(dist), dist.shape)

    # Patch bounds
    half = patch_size // 2
    y_start, y_end = y_center - half, y_center + half
    x_start, x_end = x_center - half, x_center + half

    if y_start < 0 or x_start < 0 or y_end > lat.shape[0] or x_end > lon.shape[0]:
        print("❌ Patch goes out of bounds.")
        ds.close()
        return

    # Extract each band
    patch_stack = []
    for band in rrs_band_names:
        patch = ds.variables[band][y_start:y_end, x_start:x_end]
        patch_stack.append(patch)
    patch_stack = np.stack(patch_stack, axis=0)  # shape: (bands, H, W)

    # Geo info
    res_lat = np.abs(lat[1] - lat[0])
    res_lon = np.abs(lon[1] - lon[0])
    lat_ul = lat[y_start]
    lon_ul = lon[x_start]
    transform = from_origin(lon_ul, lat_ul, res_lon, res_lat)

    # Write GeoTIFF
    with rasterio.open(
        output_path,
        "w",
        driver="GTiff",
        height=patch_stack.shape[1],
        width=patch_stack.shape[2],
        count=patch_stack.shape[0],
        dtype=patch_stack.dtype,
        crs="EPSG:4326",
        transform=transform,
    ) as dst:
        for i in range(patch_stack.shape[0]):
            dst.write(patch_stack[i], i + 1)

    print(f"✅ Saved {len(rrs_band_names)}-band Rrs patch to {output_path}")
    ds.close()


In [ ]:
product_id = 'S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_20231014T080243.SAFE'
product_groups = updated_p.groupby("product_id")

for s2_product, group in product_groups:
    if s2_product != product_id:
        continue
    for _, filament in group.iterrows():
        print(filament)
        filament_id = filament["filament_id"]
        x_center = int(filament["lat_centroid"])
        y_center = int(filament["lon_centroid"])

        extract_rrs_multiband_patch(
        "/content/hypercoast_work/output/S2A_MSI_2017_09_18_10_11_07_T32SPD_L2W.nc",
        lat_center= x_center,
        lon_center= y_center,
        patch_size=256,
        output_path= s2_product+"_"+str(group.lat_centroid)+"_"+str(group.lon_centroid,)+".tif"
        )


filament_id                                                  2000
lat_centroid                                            34.376412
lon_centroid                                            10.869906
dec_time                                              2017.713357
date                                                   2017-09-18
product_id      S2A_MSIL1C_20170918T100021_N0500_R122_T32SPD_2...
datetime                                 2017-09-18T10:11:07.848Z
mgrs_tile                                                 Unknown
Name: 0, dtype: object
